<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [ ]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=7c6899d8f61e936fcb8fae5a10255893f631641713c9b619f40ea1508d3b7066
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=879b18f958c319c53b58c5c1c9b40f66591b35be2878bbe16eba9c8808e21a5a
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [ ]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [ ]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [ ]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [ ]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [ ]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [ ]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [ ]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9810]の株価情報を取得できなかったため、スキップしました。
経過時間: 215.27072548866272秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1133100,659.000000
1,ＩＮＰＥＸ,1605,鉱業,5136000,1606.000000
2,安藤・間,1719,建設業,836500,1043.000000
3,大林組,1802,建設業,1732100,1206.000000
4,清水建設,1803,建設業,1533400,893.200012
...,...,...,...,...,...
313,エイチ・アイ・エス,9603,サービス業,1279600,2026.000000
314,エヌ・ティ・ティ・データ,9613,情報・通信業,1875900,1977.500000
315,ＳＣＳＫ,9719,情報・通信業,644800,2259.500000
316,ヤマダホールディングス,9831,小売業,2336300,422.500000


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1133100,659.000000
1,ＩＮＰＥＸ,1605,鉱業,5136000,1606.000000
2,安藤・間,1719,建設業,836500,1043.000000
3,大林組,1802,建設業,1732100,1206.000000
4,清水建設,1803,建設業,1533400,893.200012
...,...,...,...,...,...
157,東北電力,9506,電気・ガス業,2651000,881.799988
158,四国電力,9507,電気・ガス業,1181100,964.200012
159,九州電力,9508,電気・ガス業,2760000,906.700012
160,北海道電力,9509,電気・ガス業,835100,594.900024


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1133100,659.000000
1,ＩＮＰＥＸ,1605,鉱業,5136000,1606.000000
2,安藤・間,1719,建設業,836500,1043.000000
3,大林組,1802,建設業,1732100,1206.000000
4,清水建設,1803,建設業,1533400,893.200012
...,...,...,...,...,...
128,関西電力,9503,電気・ガス業,2600900,1774.000000
129,東北電力,9506,電気・ガス業,2651000,881.799988
130,九州電力,9508,電気・ガス業,2760000,906.700012
131,北海道電力,9509,電気・ガス業,835100,594.900024


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,リンクアンドモチベーション,2170,サービス業,902600,423.000000
1,パーソルホールディングス,2181,サービス業,702200,2631.000000
2,カルビー,2229,食料品,552500,2712.500000
3,明治ホールディングス,2269,食料品,619600,3254.000000
4,いちご,2337,不動産業,715300,271.000000
5,ベネフィット・ワン,2412,サービス業,593700,1520.500000
6,エムスリー,2413,サービス業,1752000,3127.000000
7,エスプール,2471,サービス業,1066800,528.000000
8,ユーグレナ,2931,食料品,534400,888.000000
9,ペッパーフードサービス,3053,小売業,555700,131.000000


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,パーソルホールディングス,2181,サービス業,702200,2631.000000
1,カルビー,2229,食料品,552500,2712.500000
2,ベネフィット・ワン,2412,サービス業,593700,1520.500000
3,エムスリー,2413,サービス業,1752000,3127.000000
4,エスプール,2471,サービス業,1066800,528.000000
5,ｇｕｍｉ,3903,情報・通信業,1017600,652.000000
6,デンカ,4061,化学,633000,2636.500000
7,大幸薬品,4574,医薬品,730500,338.000000
8,リソー教育,4714,サービス業,1371800,266.000000
9,楽天グループ,4755,サービス業,34702600,477.100006


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '2_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[1787]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1798]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1841]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1981]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4673]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5395]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6018]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6074]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6159]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6396]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6411]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7446]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7490]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7509]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7887]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8072]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,608200,2594.0
1,住石ホールディングス,1514,鉱業,528000,351.0
2,中小企業ホールディングス,1757,建設業,4342600,48.0
3,アジアゲートホールディングス,1783,建設業,393600,54.0
4,燦キャピタルマネージメント,2134,サービス業,14350900,34.0
...,...,...,...,...,...
89,ビート・ホールディングス・リミテッド,9399,情報・通信業,464852,6.0
90,ＳＥホールディングス・アンド・インキュベーションズ,9478,情報・通信業,248100,245.0
91,アゴーラ ホスピタリティー グループ,9704,サービス業,430200,25.0
92,北沢産業,9930,卸売業,305800,438.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,528000,351.0
1,中小企業ホールディングス,1757,建設業,4342600,48.0
2,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,245100,508.0
3,シダー,2435,サービス業,1591500,282.0
4,フュートレック,2468,サービス業,1461400,312.0
5,ライフドリンク カンパニー,2585,食料品,815300,3445.0
6,イメージ ワン,2667,卸売業,588100,770.0
7,コメ兵ホールディングス,2780,小売業,289600,4715.0
8,ビューティ花壇,3041,卸売業,357100,1455.0
9,アーバネットコーポレーション,3242,不動産業,386100,344.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,中小企業ホールディングス,1757,建設業,4342600,48.0
1,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,245100,508.0
2,シダー,2435,サービス業,1591500,282.0
3,フュートレック,2468,サービス業,1461400,312.0
4,ライフドリンク カンパニー,2585,食料品,815300,3445.0
5,イメージ ワン,2667,卸売業,588100,770.0
6,コメ兵ホールディングス,2780,小売業,289600,4715.0
7,ビューティ花壇,3041,卸売業,357100,1455.0
8,アーバネットコーポレーション,3242,不動産業,386100,344.0
9,クラウディアホールディングス,3607,繊維製品,766800,746.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '3_東証S_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,608200,2594.0
1,アジアゲートホールディングス,1783,建設業,393600,54.0
2,クルーズ,2138,情報・通信業,274900,864.0
3,クシム,2345,情報・通信業,225600,316.0
4,出前館,2484,情報・通信業,643900,408.0
5,ブロードバンドタワー,3776,情報・通信業,557700,145.0
6,リミックスポイント,3825,情報・通信業,680500,240.0
7,ダイトーケミックス,4366,化学,317300,585.0
8,千代田化工建設,6366,建設業,1184000,352.0
9,メディアリンクス,6659,電気機器,214000,51.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,608200,2594.0
1,クシム,2345,情報・通信業,225600,316.0
2,リミックスポイント,3825,情報・通信業,680500,240.0
3,千代田化工建設,6366,建設業,1184000,352.0
4,メディアリンクス,6659,電気機器,214000,51.0
5,ピクセラ,6731,電気機器,2281600,2.0
6,大黒屋ホールディングス,6993,小売業,766400,54.0
7,第一商品,8746,証券、商品先物取引業,240400,102.0
8,ビート・ホールディングス・リミテッド,9399,情報・通信業,464852,6.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '4_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [ ]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4450]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7699]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,465200,940.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,1462100,779.0
2,ジーエヌアイグループ,2160,医薬品,1453900,1500.0
3,ソフトフロントホールディングス,2321,情報・通信業,368100,123.0
4,イオレ,2334,サービス業,499900,2016.0
...,...,...,...,...,...
103,サンウェルズ,9229,サービス業,236200,3520.0
104,ＧＥＮＯＶＡ,9341,サービス業,276800,2024.0
105,ｉｓｐａｃｅ,9348,サービス業,951400,1686.0
106,リニューアブル・ジャパン,9522,電気・ガス業,871300,731.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ソフトフロントホールディングス,2321,情報・通信業,368100,123.0
1,イオレ,2334,サービス業,499900,2016.0
2,ジェイフロンティア,2934,食料品,229300,3500.0
3,海帆,3133,小売業,11267800,798.0
4,ロコンド,3558,小売業,260100,1760.0
5,モブキャストホールディングス,3664,情報・通信業,703600,79.0
6,ＦＦＲＩセキュリティ,3692,情報・通信業,249100,1505.0
7,アドバンスト・メディア,3773,情報・通信業,363700,1938.0
8,データセクション,3905,情報・通信業,322000,362.0
9,Ａｉｍｉｎｇ,3911,情報・通信業,742000,442.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ソフトフロントホールディングス,2321,情報・通信業,368100,123.0
1,イオレ,2334,サービス業,499900,2016.0
2,ジェイフロンティア,2934,食料品,229300,3500.0
3,海帆,3133,小売業,11267800,798.0
4,ロコンド,3558,小売業,260100,1760.0
5,ＦＦＲＩセキュリティ,3692,情報・通信業,249100,1505.0
6,アドバンスト・メディア,3773,情報・通信業,363700,1938.0
7,データセクション,3905,情報・通信業,322000,362.0
8,シェアリングテクノロジー,3989,情報・通信業,1068200,723.0
9,ティアンドエス,4055,情報・通信業,326100,2877.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '5_東証G_UPO')

In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,465200,940.0
1,メディネット,2370,サービス業,2116600,68.0
2,ディー・ディー・エス,3782,情報・通信業,579900,47.0
3,ドリコム,3793,情報・通信業,236400,727.0
4,グローバルウェイ,3936,情報・通信業,256000,218.0
5,ココナラ,4176,情報・通信業,376700,362.0
6,アクリート,4395,情報・通信業,485000,1873.0
7,ミンカブ・ジ・インフォノイド,4436,情報・通信業,253400,1760.0
8,ＢＡＳＥ,4477,情報・通信業,1353800,265.0
9,オンコセラピー・サイエンス,4564,医薬品,3546400,44.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ディー・ディー・エス,3782,情報・通信業,579900,47.0
1,ヘリオス,4593,医薬品,536600,288.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [ ]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[1787]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1798]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1841]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[1981]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4673]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5395]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6018]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6074]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6159]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6396]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6411]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7446]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7490]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7509]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7887]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7961]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8072]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,608200,2594.0
1,住石ホールディングス,1514,鉱業,528000,351.0
2,中小企業ホールディングス,1757,建設業,4342600,48.0
3,燦キャピタルマネージメント,2134,サービス業,14350900,34.0
4,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,2340000,63.0
5,シダー,2435,サービス業,1591500,282.0
6,フュートレック,2468,サービス業,1461400,312.0
7,出前館,2484,情報・通信業,643900,408.0
8,ライフドリンク カンパニー,2585,食料品,815300,3445.0
9,イメージ ワン,2667,卸売業,588100,770.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '7_東証S_選抜')

In [ ]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4450]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7699]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,1462100,779.0
1,ジーエヌアイグループ,2160,医薬品,1453900,1500.0
2,メディネット,2370,サービス業,2116600,68.0
3,フルッタフルッタ,2586,食料品,9982500,85.0
4,海帆,3133,小売業,11267800,798.0
5,ＧＡ ｔｅｃｈｎｏｌｏｇｉｅｓ,3491,不動産業,607000,1392.0
6,モブキャストホールディングス,3664,情報・通信業,703600,79.0
7,ＦＨＴホールディングス,3777,情報・通信業,4400500,38.0
8,ディー・ディー・エス,3782,情報・通信業,579900,47.0
9,ネクストジェン,3842,情報・通信業,1141900,923.0


In [ ]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2, '8_東証G_選抜')